In [406]:
import torch
import numpy as np

class X:
    def __init__(self, value, m, n):
        
        self.m = m # list of integers
        self.n = n # list of integers
        self.channels = value # list of tensors
                
    def set(self, value, m, n):
        self.m = m
        self.n = n
        self.channels = value
    
    def get(self):
        return self.channels, self.m, self.n
    
    def __str__(self):
        return 'X(channels: ' + str(self.channels.shape) +' at position: m=' + str(self.m) + ', n=' + str(self.n) + ')'
    __repr__ = __str__

    
    

class Filter(torch.nn.Module):
    def __init__(self, ms_in, ns_in, m_out, n_out, n_channels=1):
        super().__init__()
        
        self.ms_in = ms_in # list
        self.ns_in = ns_in # list
        self.m_out = m_out # single integer
        self.n_out = n_out # single integer
        self.weights = torch.autograd.Variable(torch.randn(1,n_channels,3,3))
        
    def forward(self, x_channels):
        
        #print(x.values())
    
        x_channels = torch.nn.functional.conv2d(x_channels, self.weights)
        
        #print(x.value)
        
        return x_channels
    
    def set(self, value, m_out, n_out):
        self.ms_in = None # list
        self.ns_in = None # list
        self.m_out = m_out # single integer
        self.n_out = n_out # single integer
        self.weights = value # weights in this filter
    
    def get(self):
        return self.m_out, self.n_out, self.weights
    
    def __str__(self):
        return 'DecentFilter(weights: ' + str(self.weights.shape) + ' at position: m_out=' + str(self.m_out) + ', n_out=' + str(self.n_out) + ')' + \
    '\n with inputs: ms_in= ' + ', '.join(str(m.item()) for m in self.ms_in) + ', ns_in= ' + ', '.join(str(n.item()) for n in self.ns_in) + ')'
    __repr__ = __str__
    
    
    
    
class Layer(torch.nn.Module):  
    def __init__(self, ms_in, ns_in, n_channels, n_filters):
        super().__init__()
        
        self.ms_in = ms_in
        self.ns_in = ns_in
                
        self.geometry_array = np.full(81, np.nan)
        # plus 1 here cause of to_sparse array
        self.geometry_array[0:n_filters] = range(1,n_filters+1)
        np.random.shuffle(self.geometry_array)
        self.geometry_array = self.geometry_array.reshape((9,9), order='C')
        self.geometry_array = torch.tensor(self.geometry_array)
        self.geometry_array = self.geometry_array.to_sparse(sparse_dim=2).to("cuda")

        print(self.geometry_array)
        print(self.geometry_array.values())
        
        self.filter_list = torch.nn.ModuleList([])
        for i_filter in range(n_filters):
            # minus 1 here cause of to_sparse array
            index = (self.geometry_array.values()-1 == i_filter).nonzero(as_tuple=True)[0]
            m_out = self.geometry_array.indices()[0][index]
            n_out = self.geometry_array.indices()[1][index]
            self.filter_list.append(Filter(ms_in, ns_in, m_out, n_out, n_channels))
        
    def forward(self, x):
        
        print("the 1:", x)
        
        output_list = []
        for f in self.filter_list:
            output_list.append(f(x.channels))
        
        # needs to be for each channel??
        x.channels = torch.cat(output_list, dim=1)
        # mean = torch.mean(out, 0, keepdim=True)
        
        print("the 2:", x)
        
        return x #, pos
    
    def get_filter_positions(self):
        
        ms_out = []
        ns_out = []
        for f in self.filter_list:
            ms_out.append(f.m_out)
            ns_out.append(f.n_out)
        
        return ms_out, ns_out
    
    #def __str__(self):
    #    return 'Layer(filters: )' #  + str(self.weights.shape) + ' at position: m_out=' + str(self.m_out) + ', n_out=' + str(self.n_out) + ')'
    #__repr__ = __str__
    
    
    
    
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        dim = [1, 32, 48, 64, 10]
        dim = [1, 8, 16, 24, 10]
        assert not any(i > 81 for i in dim), "filters need to be less than 81"
        
        m0 = [torch.tensor(0)]
        n0 = [torch.tensor(0)]
        #self.layer0 = Layer(m,n)
        #m,n = layer0.get_positions()
        self.layer1 = Layer(m0, n0, dim[0], dim[1])
        m1,n1 = self.layer1.get_filter_positions()
        self.layer2 = Layer(m1, n1, dim[1], dim[2])
        m2,n2 = self.layer2.get_filter_positions()
    def forward(self, x):
        # x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        return x
    
    
        
        

In [407]:
n = Net()

tensor(indices=tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
                        4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6,
                        6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8,
                        8, 8, 8, 8, 8],
                       [0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0,
                        1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1,
                        2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2,
                        3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3,
                        4, 5, 6, 7, 8]]),
       values=tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
                      nan, nan, nan, nan, nan, nan, nan, nan, nan, 6., nan, nan, nan, nan,
                      nan, nan, nan, 2., nan, nan, nan, nan, 3., 8., nan, nan, 1., nan,

In [409]:
n

Net(
  (layer1): Layer(
    (filter_list): ModuleList(
      (0): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([4], device='cuda:0'), n_out=tensor([4], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (1): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([3], device='cuda:0'), n_out=tensor([4], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (2): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([4], device='cuda:0'), n_out=tensor([0], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (3): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([5], device='cuda:0'), n_out=tensor([0], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (4): DecentFilter(weights: torch.Size([1, 1, 3, 3]) at position: m_out=tensor([6], device='cuda:0'), n_out=tensor([4], device='cuda:0'))
       with inputs: ms_in= 0, ns_in= 0)
      (5): DecentFilter(weight

In [402]:
tmp = torch.autograd.Variable(torch.randn(1, 1, 30, 30)) # one image in batch
# dense_input.shape

dense_input = X(tmp, 0, 0)

In [375]:
a = n(dense_input)

the 1: X(channels: torch.Size([1, 1, 30, 30]) at position: m=tensor(0), n=tensor(0))
the 2: X(channels: torch.Size([1, 8, 28, 28]) at position: m=tensor(0), n=tensor(0))
the 1: X(channels: torch.Size([1, 8, 28, 28]) at position: m=tensor(0), n=tensor(0))
the 2: X(channels: torch.Size([1, 16, 26, 26]) at position: m=tensor(0), n=tensor(0))


In [286]:
a

X(channels: torch.Size([1, 16, 26, 26]) at position: m=0, n=0)

In [288]:
Filter([2, 3, 5], [8, 2, 4], 4, 1)

Filter(weights: torch.Size([1, 1, 3, 3]) at position: m=4, n=1)

In [ ]:
print(a.value)

In [165]:
import numpy as np
arr = np.full(81, 0)
print(arr)
arr[0:10] = range(0,10)
print(arr)
np.random.shuffle(arr)
arr = arr.reshape((9,9), order='C')
print(arr)



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
[0 1 2 3 4 5 6 7 8 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
[[0 0 0 0 5 0 0 0 0]
 [0 0 0 0 0 0 0 2 0]
 [0 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [8 0 0 0 0 0 9 0 0]
 [0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 7]
 [0 0 0 0 0 6 0 0 0]]


In [184]:

dense_input = torch.tensor(arr)

co = dense_input.to_sparse(sparse_dim=2).to("cuda")
co

tensor(indices=tensor([[0, 1, 2, 3, 4, 4, 5, 7, 8],
                       [4, 7, 1, 6, 0, 6, 4, 8, 5]]),
       values=tensor([5, 2, 3, 1, 8, 9, 4, 7, 6]),
       device='cuda:0', size=(9, 9), nnz=9, dtype=torch.int32,
       layout=torch.sparse_coo)

In [224]:
index = (co.values() == 3).nonzero(as_tuple=True)[0]
co.indices()[0][index]
co.indices()[1][index]

In [225]:
co.indices()[0][index]

tensor([2], device='cuda:0')

In [226]:
co.indices()[1][index]

tensor([1], device='cuda:0')

In [222]:
# trying with sparse ... doesn't work cause it gets applied on all dimensions or smthg ... I want a 9x9 matrix, not a 9x9x30x30 (grid, grid, img size, img size)

In [215]:
import torch

class Filter(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
class Layer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        a = 0
    def forward(self, x):
        
        #print(x.values())
        
        values = x.values()
        w_channels = 1
        weight = torch.autograd.Variable(torch.randn(1,w_channels,3,3))
        
        #print(x.shape)
        
        values_conv = torch.nn.functional.conv2d(values, weight)
        
        #print(x.shape)
        
        x = torch.sparse_coo_tensor(x.indices(), values_conv, x.size())
        
        return x
    
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = Layer()
        self.layer1 = Layer()
        self.layer2 = Layer()
    def forward(self, x):
        # x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        return x
        
        

In [216]:
n = Net()

In [217]:
dense_input = torch.autograd.Variable(torch.randn(1,30,30)) # one image in batch
dense_input.shape

torch.Size([1, 30, 30])

In [221]:
coo_input = dense_input.to_sparse(sparse_dim=2)
coo_input

tensor(indices=tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
                         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
                         0,  0],
                       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29]]),
       values=tensor([[ 6.2641e-01,  1.3117e+00, -3.7951e-01,  4.3884e-01,
                        2.7492e-01,  1.4680e+00,  1.7618e-01, -5.8837e-02,
                        3.7669e-01,  6.6832e-02, -8.3039e-01, -9.2211e-01,
                        3.6151e-01, -5.6165e-01, -6.5651e-01, -2.5526e-01,
                       -1.2958e+00, -3.1554e-01,  4.9020e-01, -4.6814e-01,
                        1.4990e-01, -1.6607e+00, -1.0564e+00,  4.8079e-01,
                       -1.7671e+00, -2.0465e+00,  2.9619e-01, -4.3248e-01,
                        2.1706e+00,  6.6934e-02],
                      [ 6.3521e-02,  

In [219]:
# coo_input.values()

In [220]:
a = n(coo_input)

RuntimeError: values has incorrect size, expected [1, 30, 30], got [1, 28, 28]